# Interpretation of PCA

One of the most significant benefits of PCA over other dimensionality reduction approaches is that the results of PCA can be interpreted. 
Here we will look at interpretating the principal components from some data, specifically the breast cancer data highlighted previously. 
We can read in the data, scale it and perform the PCA analysis. 

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

data = pd.read_csv('./../data/breast-cancer.csv')
scaled_data = StandardScaler().fit_transform(data[data.columns[1:]])
pca = PCA()
transformed = pca.fit_transform(scaled_data)
pc = pd.DataFrame(transformed, columns=['PC{}'.format(i + 1) for i in range(transformed.shape[1])])
pc['Diagnosis'] = data['Diagnosis']
pc.head()

## How Many Principal Components to Investigate?

A common question with many dimensionality reduction algorithms centres on the idea of how many dimensions do we want to end up with?
The answer to this question is usually very problem specific and depends on what information you want to gain about your data. 
However, for PCA, the scree plot is a useful tool to help us visualise the information present in each component. 

The scree or elbow plot involves plotting the explained variance (or explained variance ratio) as a function of components. 
So for the breast cancer dataset:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, '.-')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Proportion of Variance Explained')
plt.show()

The aim of the scree or elbow plot is to identify the "elbow" in the dataset. 
This is a subjective approach, but the elbow is essentially the point at which adding more components doesn't not add a significant gain in explained variance. 
So for the plot above, I would say that the elbow is around three principal components, so the first two should be sufficient for most interpretations. 

A slightly older approach is to use principal components that have eigenvalues greater than one, recall that the eigenvalue of the covariance matrix is the explained variance. 
We can plot this so called Kaiser criterion (or KL1) on our scree plot to see if this agrees with our suggestion from the elbow method. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(1, len(pca.explained_variance_) + 1), pca.explained_variance_, '.-')
ax.axhline(1, color='#ff7f0e', linestyle='--', label='KL Criterion')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Proportion of Variance Explained')
ax.legend()
plt.show()

## What Do the Principal Components Mean?

It is possible to use the principal components and their eigenvalues to derive an interpretation of the data. 
This can be extremely powerful in understanding the terns and relationship that lead to our observations. 
The first interpretative tool is the loading matrix, this gives a quantitative description of how each feature contributes to a given principal component. 
The loading of the *n*th principal component is the *n*-th row of the component matrix (where the principal components themselves are the columns) multiplied by the *n*th eigenvalue.
The code below calculates the loading matrix for the first two principal components and plots them against the relevant features. 

In [ ]:
import numpy as np

loading_matrix = pd.DataFrame(pca.components_[:2].T * np.sqrt(pca.explained_variance_[:2]), 
                              columns=['PC1', 'PC2'], index=data.columns[1:11])

fig, ax = plt.subplots(figsize=(6, 4))
loading_matrix.plot(kind='bar', ax=ax)
ax.set_xlabel('Features')
ax.set_ylabel('Loading')
plt.show()

We can see from the above plot, that the first principal component contains significant positive loading from most of the features. 
This means that a positive value for the transformed data into the first principal component is associated with a positive value for all of the features. 
On the other hand, the second principal component has a negative loading for many of the features (radius, texture, etc.). 
This means that a positive value for the transformed data into the second principal component is associated with a negative value for these features. 

Let's look at the data transformed by the first and second principal components, we have coloured this data depending on whether the cancer was found to be malignant or benign. 
````{margin}
```{note}
Here, the `seaborn` library is used to create a nice statistical plot. 
Seaborn is a nice wrapper around the `matplotlib` library that makes creating common statistical plots easier. 
```
````

In [ ]:
import seaborn as sns

sns.jointplot(x='PC1', y='PC2', hue='Diagnosis', data=pc, kind='scatter')
plt.show()

From this it is clear that benign tumours typically had less of principal component 1. 
Looking at this in the context of the loading matrix plot above, we can rationalise that, typically, malignant tumours have larger values of the features, such as concave points, than benign tumours.
One could imagine using a clustering algorithm on principal component 1, to classify new data is benign or malignant. 
Principal component 2 is harder to interpret, as the distinction is less clear.
This is expected, given that principal component 2 explains only 25 % of the variance, compared against principal component 1's 60 %. 